# FINAL PROJECT ITS365 by Abbie McDowell and Oona Kintscher

Goal: Predict NBA players win shares for their age 25 season based on their rookie years statistics.

## Import Libraries

In [2]:

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, VarianceThreshold
from sklearn.metrics import mean_absolute_error, r2_score

from keras.layers import Normalization, Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras import regularizers

from keras.callbacks import ModelCheckpoint
from keras import backend as K


## Import Data from .csv File

In [3]:

df = pd.read_csv(r'365finalproject.csv', encoding= 'unicode_escape')


In [4]:
df

,PlayerID,Player,Age,Games,Minutes,FGM,FGA,3PM,3PA,FTM,...,PF,PTS,FG%,3P%,FT%,MPG,PPG,RPG,APG,WS_25
0,duartch01,Chris Duarte,24,55,1541,268,621,94,255,90,...,95,720,0.432,0.369,0.804,28.0,13.1,4.1,2.1,0.0
1,marshna01,Naji Marshall,23,32,700,82,209,29,83,53,...,58,246,0.392,0.349,0.707,21.9,7.7,4.6,2.8,2.5
2,pritcpa01,Payton Pritchard,23,66,1268,184,418,102,248,40,...,104,510,0.440,0.411,0.889,19.2,7.7,2.4,1.8,0.6
3,windldy01,Dylan Windler,24,31,513,57,130,26,77,21,...,37,161,0.438,0.338,0.778,16.5,5.2,3.5,1.1,0.8
4,clarkbr01,Brandon Clarke,23,58,1300,296,479,23,64,85,...,100,700,0.618,0.359,0.759,22.4,12.1,5.9,1.4,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,kunneke01,Kevin Kunnert,22,64,701,105,215,0,0,21,...,151,231,0.488,0.000,0.636,11.0,3.6,3.4,0.7,4.1
1032,mcneila01,Larry McNeill,23,54,516,106,220,0,0,99,...,76,311,0.482,0.000,0.707,9.6,5.8,2.7,0.4,4.3
1033,ratleed01,Ed Ratleff,23,81,1773,254,585,0,0,103,...,182,611,0.434,0.000,0.798,21.9,7.5,3.5,2.2,4.5
1034,wattssl01,Slick Watts,22,62,1424,198,510,0,0,100,...,207,496,0.388,0.000,0.645,23.0,8.0,2.9,5.7,3.3


In [5]:

names_ds = df[["PlayerID", "Player"]]
features_ds_raw = df.loc[:,'Age':'APG']
features_ds_raw = features_ds_raw.drop(["3PM", "3PA", "3P%"], axis=1)
labels_ds_raw = df[["WS_25"]]


In [6]:
names_ds

,PlayerID,Player
0,duartch01,Chris Duarte
1,marshna01,Naji Marshall
2,pritcpa01,Payton Pritchard
3,windldy01,Dylan Windler
4,clarkbr01,Brandon Clarke
...,...,...
1031,kunneke01,Kevin Kunnert
1032,mcneila01,Larry McNeill
1033,ratleed01,Ed Ratleff
1034,wattssl01,Slick Watts


In [7]:
features_ds_raw

,Age,Games,Minutes,FGM,FGA,FTM,FTA,ORB,TRB,AST,STL,BLK,PF,PTS,FG%,FT%,MPG,PPG,RPG,APG
0,24,55,1541,268,621,90,112,41,226,114,56,10,95,720,0.432,0.804,28.0,13.1,4.1,2.1
1,23,32,700,82,209,53,75,15,148,88,26,10,58,246,0.392,0.707,21.9,7.7,4.6,2.8
2,23,66,1268,184,418,40,45,33,158,120,37,9,104,510,0.440,0.889,19.2,7.7,2.4,1.8
3,24,31,513,57,130,21,27,20,107,33,19,12,37,161,0.438,0.778,16.5,5.2,3.5,1.1
4,23,58,1300,296,479,85,112,92,345,81,32,48,100,700,0.618,0.759,22.4,12.1,5.9,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,22,64,701,105,215,21,33,83,217,43,10,54,151,231,0.488,0.636,11.0,3.6,3.4,0.7
1032,23,54,516,106,220,99,140,60,146,24,35,6,76,311,0.482,0.707,9.6,5.8,2.7,0.4
1033,23,81,1773,254,585,103,129,93,286,181,90,27,182,611,0.434,0.798,21.9,7.5,3.5,2.2
1034,22,62,1424,198,510,100,155,72,182,351,115,13,207,496,0.388,0.645,23.0,8.0,2.9,5.7


In [8]:
labels_ds_raw

,WS_25
0,0.0
1,2.5
2,0.6
3,0.8
4,6.3
...,...
1031,4.1
1032,4.3
1033,4.5
1034,3.3


## Normalization of Feature Data

In [9]:

feature_normalization_layer = tf.keras.layers.Normalization(axis=-1)
feature_normalization_layer.adapt(features_ds_raw)


In [10]:

features_ds_normalized = feature_normalization_layer(features_ds_raw)


In [11]:
features_ds = np.array(features_ds_normalized)

In [12]:
features_ds

array([[ 1.587192  , -1.133756  ,  0.03314409, ...,  0.9669427 ,
         0.09333976,  0.11713215],
       [ 0.8004314 , -2.9718945 , -1.2081058 , ..., -0.21646269,
         0.3200158 ,  0.55341506],
       [ 0.8004314 , -0.2546463 , -0.36978245, ..., -0.21646269,
        -0.6773586 , -0.06984619],
       ...,
       [ 0.8004314 ,  0.94413966,  0.37555787, ..., -0.26029247,
        -0.17867143,  0.17945836],
       [ 0.01367071, -0.5743225 , -0.13953872, ..., -0.1507179 ,
        -0.4506826 ,  2.3608725 ],
       [ 0.8004314 , -0.33456534, -0.44653037, ..., -0.23837757,
         1.0000439 , -0.81775963]], dtype=float32)

## Normalization of Label Data

In [13]:
epsilon = 0.0001

In [14]:

labels_column_means = labels_ds_raw.mean(axis=0)
labels_column_std = labels_ds_raw.std(0) + epsilon


In [15]:
labels_column_means

WS_25    4.16583
dtype: float64

In [16]:
labels_column_std

WS_25    3.307225
dtype: float64

In [21]:

labels_ds_normalized = (labels_ds_raw - labels_column_means) / labels_column_std


In [22]:
labels_ds = np.array(labels_ds_normalized)

In [23]:
labels_ds

array([[-1.25961514],
       [-0.50369429],
       [-1.07819413],
       ...,
       [ 0.10104239],
       [-0.26179962],
       [-0.53393112]])

In [24]:

def denormalize(input_label_normalized):
    
    input_label_raw = (input_label_normalized * labels_column_std[0]) + labels_column_means[0] 
    
    return input_label_raw
    

## K Fold Validation with 20Folds

In [25]:
from sklearn.model_selection import KFold

In [26]:
kf = KFold(n_splits=10)

In [27]:

mae_list = []
r2_list = []


In [28]:

best_r2 = -np.inf
best_weights = None


In [105]:
y = labels_ds

In [106]:
X = features_ds_pca_kmeans

In [107]:

model = Sequential()

model.add(Dense(units=1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(units=512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
    
model.add(Dense(units=1, activation='linear'))


In [108]:

callback = tf.keras.callbacks.EarlyStopping (
        monitor = 'mean_absolute_error',
        patience = 5,
        restore_best_weights = True
)


In [111]:
model.load_weights('Model-0.69.h5')

In [112]:

for train_index, test_index in kf.split(X):
    
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics=['mean_absolute_error'])
    
    model.fit(
        x_train, 
        y_train, 
        epochs=50, 
        callbacks = [callback], 
        validation_data=(x_test, y_test)
    )
    
    y_pred_normalized = model.predict(x_test)
    
    mae = mean_absolute_error(y_test, y_pred_normalized)
    r2 = r2_score(y_test, y_pred_normalized)
    
    if r2 > best_r2:
        best_r2 = r2
        best_model = model
    
    mae_list.append(mae)
    r2_list.append(r2)
    

Epoch 1/50
30/30 [==============================] - 2s 28ms/step - loss: 0.6290 - mean_absolute_error: 0.5633 - val_loss: 0.4247 - val_mean_absolute_error: 0.4432
Epoch 2/50
30/30 [==============================] - 1s 19ms/step - loss: 0.6193 - mean_absolute_error: 0.5552 - val_loss: 0.4273 - val_mean_absolute_error: 0.4454
Epoch 3/50
30/30 [==============================] - 1s 22ms/step - loss: 0.6052 - mean_absolute_error: 0.5470 - val_loss: 0.4283 - val_mean_absolute_error: 0.4463
Epoch 4/50
30/30 [==============================] - 1s 20ms/step - loss: 0.6100 - mean_absolute_error: 0.5515 - val_loss: 0.4329 - val_mean_absolute_error: 0.4525
Epoch 5/50
30/30 [==============================] - 1s 21ms/step - loss: 0.6182 - mean_absolute_error: 0.5633 - val_loss: 0.4274 - val_mean_absolute_error: 0.4463
Epoch 6/50
30/30 [==============================] - 1s 21ms/step - loss: 0.5993 - mean_absolute_error: 0.5503 - val_loss: 0.4285 - val_mean_absolute_error: 0.4469
Epoch 7/50
30/30 [====

Epoch 11/50
30/30 [==============================] - 1s 21ms/step - loss: 0.5558 - mean_absolute_error: 0.5264 - val_loss: 0.5453 - val_mean_absolute_error: 0.5293
Epoch 12/50
4/4 [==============================] - 0s 5ms/step
Epoch 1/50
30/30 [==============================] - 2s 29ms/step - loss: 0.5676 - mean_absolute_error: 0.5228 - val_loss: 0.5062 - val_mean_absolute_error: 0.4986
Epoch 2/50
30/30 [==============================] - 1s 19ms/step - loss: 0.5408 - mean_absolute_error: 0.5154 - val_loss: 0.5157 - val_mean_absolute_error: 0.5035
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.5450 - mean_absolute_error: 0.5188 - val_loss: 0.5206 - val_mean_absolute_error: 0.5049
Epoch 4/50
30/30 [==============================] - 1s 19ms/step - loss: 0.5505 - mean_absolute_error: 0.5188 - val_loss: 0.5227 - val_mean_absolute_error: 0.5086
Epoch 5/50
30/30 [==============================] - 1s 21ms/step - loss: 0.5474 - mean_absolute_error: 0.5176 - val_loss:

In [113]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 1024)              22528     
                                                                 
 dropout_8 (Dropout)         (None, 1024)              0         
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 256)               131328    
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 128)              

In [114]:

mean_mae = np.mean(mae_list)
print("Mean MAE:", mean_mae)

mean_r2 = np.mean(r2_list)
print("Mean R2:", mean_r2)


Mean MAE: 0.511195436026916
Mean R2: 0.5605325438453214


In [115]:

best_mae = np.min(mae_list)
print("Best MAE:", best_mae)

best_r2 = np.max(r2_list)
print("Best R2:", best_r2)


Best MAE: 0.4558139085953271
Best R2: 0.6722055934478701


In [ ]:
best_model.save_weights('Model-0.69.h5')

In [ ]:

n_train1, n_test1, x_train1, x_test1, y_train1, y_test1 = train_test_split(names_ds, X, y, test_size=0.3)


### R2 on Entire Dataset (Normalized)

In [116]:
model.evaluate(X, y)

33/33 [==============================] - 0s 5ms/step - loss: 0.4850 - mean_absolute_error: 0.4800


[0.4850108325481415, 0.4800297021865845]

In [117]:
y_pred_normalized = model.predict(X)

33/33 [==============================] - 0s 4ms/step


In [121]:

mae_normalized = mean_absolute_error(labels_ds, y_pred_normalized)
print(mae_normalized)

r2_score_normalized = r2_score(labels_ds, y_pred_normalized)
print(r2_score_normalized)


0.4800297125707189
0.6326198439323807


### R2 on Entire Dataset (Raw)

In [119]:
y_pred_raw = denormalize(model.predict(X))

33/33 [==============================] - 0s 3ms/step


In [120]:

mae_raw = mean_absolute_error(labels_ds_raw, y_pred_raw)
print(mae_raw)

r2_score_raw = r2_score(labels_ds_raw, y_pred_raw)
print(r2_score_raw)


1.5875660567209986
0.6326198419200653


# Data Visualization 

## Histogram

In [ ]:
sb.histplot(data=features_ds_raw)
plt.legend(features_ds_raw.head(), bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sb.histplot(data=features_ds)
plt.legend(features_ds_raw.head(), bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sb.histplot(data=features_ds_K_Means)
plt.legend(np.arange(features_ds_K_Means.shape[1]), bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sb.histplot(data=features_ds_pca)
plt.legend(np.arange(features_ds_pca.shape[1]), bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

# Boxplot

In [ ]:
plt.rcParams.update({'font.size': 15})
fig, ax = plt.subplots(figsize=(30, 20))
sb.boxplot(data=features_ds_raw, ax=ax)
plt.show()

## Correlation Matrix

In [ ]:

C_mat = features_ds_raw.corr()
C_mat


In [ ]:

plt.rcParams.update({'font.size': 8})
fig = plt.figure(figsize = (8,8))
sb.heatmap(C_mat, vmax=.8, square=True)
plt.show()


## Filter Feature Selection Techniques

### Univariate Feature Selection

In [ ]:

f_val, p_val = f_regression(features_ds, np.array(labels_ds).ravel())


In [ ]:
f_val

In [ ]:
p_val

In [ ]:

feature_dictonary={'features':features_ds_raw.columns.tolist(), 'f_scores':f_val.tolist()}
features_f_score= pd.DataFrame(feature_dictonary).sort_values(by='f_scores', ascending=False).reset_index(drop=True)


In [ ]:
features_f_score

### Variance Threshold

In [ ]:

variance_filter = VarianceThreshold(threshold=0.25)
features_variance_selected = variance_filter.fit_transform(features_ds_raw)


In [ ]:
pd.DataFrame(features_variance_selected)

## Principal Component Analysis

In [29]:
from sklearn.decomposition import PCA

In [30]:
pca_model = PCA()

In [31]:
pca_model.fit(features_ds)

PCA()

In [32]:
features_ds_pca = pca_model.transform(features_ds)

In [33]:
pd.DataFrame(features_ds_pca)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.292134,-1.257798,-0.707501,-1.869496,1.177831,0.822886,0.089581,-0.574167,0.762674,0.344767,-0.210416,-0.312725,0.262175,0.281941,0.023277,-0.140330,0.026398,-0.086339,-0.027676,-0.024713
1,-3.327180,-0.757063,-1.204306,-0.415835,2.399911,1.717979,-0.156369,-0.177034,-0.333082,-0.166262,0.257170,-0.175745,0.384300,0.161141,0.093361,0.114670,-0.033822,0.139177,0.097603,0.020054
2,-2.015606,-1.362153,-0.405446,-1.433654,-0.467715,0.061849,-0.941400,-0.417769,0.592103,-0.012417,-0.298907,0.058895,0.036770,0.066277,0.038942,-0.115642,0.097798,-0.025997,0.074460,-0.040713
3,-4.596894,-0.145407,-0.969570,-1.644120,1.998151,1.010000,-0.636268,-0.196166,-0.066107,0.347687,0.055709,-0.046553,0.124933,-0.300156,-0.063901,0.008944,0.077846,0.047546,0.057027,-0.015374
4,-0.049892,1.643681,-0.343754,-1.483563,1.923791,-1.813028,-1.181992,-0.583150,1.041105,0.060921,-0.775561,0.043405,0.367791,0.211550,-0.012893,-0.105319,0.023094,0.008346,-0.004242,0.040277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,-3.179523,1.807039,0.546411,0.404087,0.054136,-0.250871,-0.225722,0.239025,0.294486,-0.461221,0.055090,0.583488,-0.024619,-0.018336,-0.049239,0.009372,-0.107070,-0.003852,0.028336,0.000166
1032,-3.357408,0.636947,-0.259547,-1.023044,0.701822,-0.669717,0.327507,0.246211,-0.920763,0.521672,-0.149619,0.217559,0.125018,-0.079194,-0.044589,-0.069223,0.037971,-0.083109,0.034924,0.066822
1033,0.475405,-0.951874,1.057638,-0.566269,-0.912363,0.382992,-0.458896,-0.299393,0.093178,0.560132,-0.148677,0.032868,0.021519,-0.021933,-0.029983,0.072934,0.009080,-0.007498,-0.040534,0.011361
1034,0.281951,-2.579041,1.511082,1.600706,1.212078,1.070659,-0.014405,0.085989,-0.765637,0.081333,0.780976,0.645499,-0.155930,0.169430,0.200307,0.137249,-0.056131,-0.002074,0.065931,0.025794


## K-Means Clustering

In [34]:
from sklearn.cluster import KMeans

In [35]:
k_means_model = KMeans(n_init='auto')

In [36]:
k_means_model.fit(features_ds)

C:\Users\oonak\Anaconda3\envs\ITS365\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


KMeans(n_init='auto')

In [37]:
features_ds_group = k_means_model.predict(features_ds)

In [38]:
features_ds_group = features_ds_group.reshape(features_ds_group.size,1)

In [39]:
features_ds_group.shape

(1036, 1)

In [40]:
features_ds.shape

(1036, 20)

In [41]:
features_ds_K_Means = np.concatenate((features_ds, features_ds_group), axis=-1)

In [42]:
pd.DataFrame(features_ds_K_Means)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.587192,-1.133756,0.033144,0.185949,0.301294,-0.327468,-0.445468,-0.733264,-0.279039,-0.182255,...,-0.626633,-0.957988,0.261620,-0.450921,0.951292,0.868568,0.966943,0.093340,0.117132,1.0
1,0.800431,-2.971895,-1.208106,-1.071918,-1.019095,-0.751425,-0.763925,-1.119656,-0.712215,-0.392760,...,-0.626633,-1.512201,-0.986292,-1.233555,-0.139663,0.061195,-0.216463,0.320016,0.553415,0.0
2,0.800431,-0.254646,-0.369782,-0.382120,-0.349286,-0.900383,-1.022133,-0.852154,-0.656680,-0.133676,...,-0.652456,-0.823179,-0.291252,-0.294395,1.907283,-0.296166,-0.216463,-0.677359,-0.069846,0.0
3,1.587192,-3.051814,-1.484103,-1.240986,-1.272277,-1.118090,-1.177058,-1.045350,-0.939910,-0.838061,...,-0.574986,-1.826754,-1.210074,-0.333526,0.658871,-0.653528,-0.764336,-0.178671,-0.506129,0.0
4,0.800431,-0.893999,-0.322553,0.375305,-0.153792,-0.384760,-0.445468,0.024659,0.381833,-0.449435,...,0.354643,-0.883094,0.208966,3.188325,0.445179,0.127374,0.747794,0.909374,-0.319151,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,0.013671,-0.414484,-1.206630,-0.916375,-0.999866,-1.118090,-1.125417,-0.109092,-0.329021,-0.757097,...,0.509581,-0.119179,-1.025783,0.644766,-0.938197,-1.381486,-1.114974,-0.224007,-0.755434,3.0
1032,0.800431,-1.213675,-1.479675,-0.909612,-0.983842,-0.224344,-0.204473,-0.450901,-0.723322,-0.910929,...,-0.729925,-1.242584,-0.815165,0.527371,-0.139663,-1.566785,-0.632846,-0.541353,-0.942412,3.0
1033,0.800431,0.944140,0.375558,0.091271,0.185920,-0.178510,-0.299150,0.039520,0.054174,0.360203,...,-0.187641,0.345161,-0.025347,-0.411790,0.883810,0.061195,-0.260292,-0.178671,0.179458,1.0
1034,0.013671,-0.574323,-0.139539,-0.287442,-0.054442,-0.212885,-0.075369,-0.272566,-0.523395,1.736588,...,-0.549163,0.719630,-0.328110,-1.311818,-0.836975,0.206787,-0.150718,-0.450683,2.360873,1.0


In [43]:
features_ds_pca_kmeans = np.concatenate((features_ds_pca, features_ds_group), axis=-1)

In [44]:
pd.DataFrame(features_ds_pca_kmeans)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,-0.292134,-1.257798,-0.707501,-1.869496,1.177831,0.822886,0.089581,-0.574167,0.762674,0.344767,...,-0.312725,0.262175,0.281941,0.023277,-0.140330,0.026398,-0.086339,-0.027676,-0.024713,1.0
1,-3.327180,-0.757063,-1.204306,-0.415835,2.399911,1.717979,-0.156369,-0.177034,-0.333082,-0.166262,...,-0.175745,0.384300,0.161141,0.093361,0.114670,-0.033822,0.139177,0.097603,0.020054,0.0
2,-2.015606,-1.362153,-0.405446,-1.433654,-0.467715,0.061849,-0.941400,-0.417769,0.592103,-0.012417,...,0.058895,0.036770,0.066277,0.038942,-0.115642,0.097798,-0.025997,0.074460,-0.040713,0.0
3,-4.596894,-0.145407,-0.969570,-1.644120,1.998151,1.010000,-0.636268,-0.196166,-0.066107,0.347687,...,-0.046553,0.124933,-0.300156,-0.063901,0.008944,0.077846,0.047546,0.057027,-0.015374,0.0
4,-0.049892,1.643681,-0.343754,-1.483563,1.923791,-1.813028,-1.181992,-0.583150,1.041105,0.060921,...,0.043405,0.367791,0.211550,-0.012893,-0.105319,0.023094,0.008346,-0.004242,0.040277,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,-3.179523,1.807039,0.546411,0.404087,0.054136,-0.250871,-0.225722,0.239025,0.294486,-0.461221,...,0.583488,-0.024619,-0.018336,-0.049239,0.009372,-0.107070,-0.003852,0.028336,0.000166,3.0
1032,-3.357408,0.636947,-0.259547,-1.023044,0.701822,-0.669717,0.327507,0.246211,-0.920763,0.521672,...,0.217559,0.125018,-0.079194,-0.044589,-0.069223,0.037971,-0.083109,0.034924,0.066822,3.0
1033,0.475405,-0.951874,1.057638,-0.566269,-0.912363,0.382992,-0.458896,-0.299393,0.093178,0.560132,...,0.032868,0.021519,-0.021933,-0.029983,0.072934,0.009080,-0.007498,-0.040534,0.011361,1.0
1034,0.281951,-2.579041,1.511082,1.600706,1.212078,1.070659,-0.014405,0.085989,-0.765637,0.081333,...,0.645499,-0.155930,0.169430,0.200307,0.137249,-0.056131,-0.002074,0.065931,0.025794,1.0


In [45]:
features_ds_all = np.concatenate((features_ds, features_ds_pca, features_ds_group), axis=-1)

In [46]:
pd.DataFrame(features_ds_all)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.587192,-1.133756,0.033144,0.185949,0.301294,-0.327468,-0.445468,-0.733264,-0.279039,-0.182255,...,-0.312725,0.262175,0.281941,0.023277,-0.140330,0.026398,-0.086339,-0.027676,-0.024713,1.0
1,0.800431,-2.971895,-1.208106,-1.071918,-1.019095,-0.751425,-0.763925,-1.119656,-0.712215,-0.392760,...,-0.175745,0.384300,0.161141,0.093361,0.114670,-0.033822,0.139177,0.097603,0.020054,0.0
2,0.800431,-0.254646,-0.369782,-0.382120,-0.349286,-0.900383,-1.022133,-0.852154,-0.656680,-0.133676,...,0.058895,0.036770,0.066277,0.038942,-0.115642,0.097798,-0.025997,0.074460,-0.040713,0.0
3,1.587192,-3.051814,-1.484103,-1.240986,-1.272277,-1.118090,-1.177058,-1.045350,-0.939910,-0.838061,...,-0.046553,0.124933,-0.300156,-0.063901,0.008944,0.077846,0.047546,0.057027,-0.015374,0.0
4,0.800431,-0.893999,-0.322553,0.375305,-0.153792,-0.384760,-0.445468,0.024659,0.381833,-0.449435,...,0.043405,0.367791,0.211550,-0.012893,-0.105319,0.023094,0.008346,-0.004242,0.040277,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,0.013671,-0.414484,-1.206630,-0.916375,-0.999866,-1.118090,-1.125417,-0.109092,-0.329021,-0.757097,...,0.583488,-0.024619,-0.018336,-0.049239,0.009372,-0.107070,-0.003852,0.028336,0.000166,3.0
1032,0.800431,-1.213675,-1.479675,-0.909612,-0.983842,-0.224344,-0.204473,-0.450901,-0.723322,-0.910929,...,0.217559,0.125018,-0.079194,-0.044589,-0.069223,0.037971,-0.083109,0.034924,0.066822,3.0
1033,0.800431,0.944140,0.375558,0.091271,0.185920,-0.178510,-0.299150,0.039520,0.054174,0.360203,...,0.032868,0.021519,-0.021933,-0.029983,0.072934,0.009080,-0.007498,-0.040534,0.011361,1.0
1034,0.013671,-0.574323,-0.139539,-0.287442,-0.054442,-0.212885,-0.075369,-0.272566,-0.523395,1.736588,...,0.645499,-0.155930,0.169430,0.200307,0.137249,-0.056131,-0.002074,0.065931,0.025794,1.0


## Preprocessing for Model 

In [47]:
def preprocessing(input_sample):
    
    sample_cluster = k_means_model.predict(input_sample)
    sample_pca = pca_model.transform(input_sample)
    
    sample_label = np.concatenate((sample_pca, sample_cluster), axis=-1)

    return sample_label

## Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
mean_abs_error = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print("Mean Absolute Error:", mean_abs_error)
print("R-squared:", r2)